>Mobile Communications - FEUP - Network Simulation - WiFi

# Objectives of the Project
- To study the performance of wireless network (WiFi) scenarios 
using discrete event simulation techniques
- Using the network simulator [ns-3](https://www.nsnam.org/)

## Install and start using ns-3
- Install VMware or VirtualBox in your computer
- Install Ubuntu as a virtual machine and build ns-3 within Ubuntu 
- You may use 2 alternatives
1. Download and run the Ubuntu-COMO image available at the netlab (via moodle)
    - includes complied ns-3 software and examples
2. Do it by yourself
    - Download the latest version of Ubuntu and update it
        `sudo apt-get update ; sudo apt-get build-essential`
    - Install Python
    - Build ns-3 according to the ns-3 installation recommendations
    - Copy the provided examples (available in Moodle) to the scratch folder 
- Study the provided examples (one example for each study)
- Additionally, you may also study other ns-3 tutorials
- Look at the ns-3 modules related to our course: 
    - network, applications, wifi, propagation, mobility
- Start your study by using the provided examples

## __First Study - Throughput vs Distance__ 

---
### 1 AP, 1 STA
---
- Assume that both the AP and the STA have full TCP/IP stacks
- It simplifies the simulation script
- Let the STA generate one TCP flow towards the AP
- Carry out simulations for different distances between the STA and the AP
- For each distance measure the average throughput obtained by the flow
-Throughput (in bit/s) is measured at the node that receiving the information 
- Use both flowmon and wireshark to characterize throughputs Flowmon enables you to obtain relevant statistics, including histograms Wireshark enables you to observe packets along the time, including the TCP congestion control mechanism in action
- Represent results as a  function of distance
First Study -Throughput vs distance
---
Notes:

- You may start at 100m. Lower `distances` result in about the same effective TCP throughput. 
- The maximum link `distance` is expected to be around 1450m. At 1500m there is no connectivity.
- You may collect results with steps of 100m between 100m and 1500m. Decrease the step size if more resolution is needed for the resulting plot($s$).

The default settings are:
> ```duration=10``` ($s$)
>
> ```distance=100``` ($m$)

You may change the parameters directly in the code and run the scenario without any argument:

```./waf --run "scratch/first_study"```

or feed the `distance` ($m$) between nodes and the `duraton` of the flow ($s$) you want to simulate via command line arguments:

```./waf --run "scratch/first_study --distance=100 --duration=60"```


In [ ]:
import subprocess
import re
import pandas as pd
import numpy as np
import plotly.express as px

# Defining variables to use for the study
default = True

print('First Study\n')

if(default): # 
    start = 100
    stop = 1500
    step = 100
    duration = 1
    distance = start
else:
    start = input('Limite Min?')
    stop = input('Limite Max?')
    step = input('Passo?')
    duration = input('Duracao?')
    

regex = r"       Throughput: (\d+\.\d+) Kbps"

rx = re.compile(regex)

df = pd.DataFrame(columns=['Flow1','Flow2','Distance'],dtype= np.float64)

for distance in range(start,stop,step):
    i = int (distance - start)/step 
    print(f'Distance is {distance}m, iteration {i}, running command:\n    ./waf --run "scratch/first_study --distance={distance} --duration={duration}"')
    process = subprocess.run([f'./waf --run "scratch/first_study --distance={distance} --duration={duration}"'], shell=True, check=True, stdout=subprocess.PIPE, universal_newlines=True)
    output = rx.findall(process.stdout)
    df.loc[i] = output+[distance]
    print(f'Throughput Results:\n    Flow 1 -> {output[0]} Kbps\n    Flow 2 -> {output[1]} Kbps\n\n')



fig = px.line(df, x="Distance", y=["Flow1","Flow2"], title='Throughput vs Distance')
fig.show()

## __Second Study - Throughput vs number of STAs__

---
### 1 AP, n STA
---

 Assume again that both the AP and the STAs have full TCP/IP stacks
- Each STA generates one TCP flow  towards the AP
- Carry out simulations for variable numbers of STA located around the AP
- same distance for all STA, circumference around AP
- For each number of STAs measure the throughput obtained by stations
- aggregated (sum of all stations) and average
- Represent results as a function of number of STAs
---
- The coded scenario starts each flow to the sink node between t=2.0s and t=4.0s of simulation time, to avoid collisions.
- Try to simulate using longer `durations` so that the TCP throughput of each flow has a chance of stabilizing.
- You may simulate, for example, up to 15 nodes. When increasing the number of senders, some TCP connection establishments may timeout, leaving some senders idle throughout the simulation.

The default settings are:
> ```duration=10``` ($s$)
>
> ```nr_of_senders=2``` ($nodes$)

You may change the parameters directly in the code and run the scenario without any argument:

```./waf --run "scratch/second_study"```

or feed the `duration` of each flow ($s$) and the `number` of sender nodes you want to simulate via command line arguments:

```./waf --run "scratch/second_study --duration=60  --nr_of_senders=10"```


In [ ]:
import subprocess
import re
import pandas as pd
import numpy as np

# Defining variables to use for the study
default = True


print('Second Study\n')

if(default): # 
    start = 1
    stop = 5
    step = 1
    duration = 1
else:
    start = input('Limite Min?')
    stop = input('Limite Max?')
    step = input('Passo?')
    duration = input('Duracao?')

regex = r"       Throughput: (\d+\.\d+) Kbps"
rx = re.compile(regex)

df = pd.DataFrame(columns=['Nodes'],dtype= np.float64)

for nodes in range(start,stop+step,step):
    i = int (nodes- start)/step 
    print(f'Duration is {duration}m, nodes {nodes}, running command:\n    ./waf --run "scratch/second_study --duration={duration} --nr_of_senders={nodes}"')
    process = subprocess.run([f'./waf --run "scratch/second_study --duration={duration} --nr_of_senders={nodes}"'], shell=True, check=True, stdout=subprocess.PIPE, universal_newlines=True)
    output = rx.findall(process.stdout)
    print(process.stdout)
    print(output)
    #df.loc[i] = [nodes]+output    
    #print(f'Throughput Results:\n    Flow 1 -> {output[0]} Kbps\n    Flow 2 -> {output[1]} Kbps\n\n')


## __Third Study - TCP Throughput vs UDP bitrate__

---
### 1 AP , 2 STA
---
- One STA generates one TCP flow towards the AP
- The other STA generates a Constant BitRate UDP flow towards the AP 
- two ports in AP, one for each STA
- Same distance for both STAs; the nodes can listen to each other
- Carry out simulations for different bitrates generated by the UDP STA
- Measure throughputs, delays and packet losses
- Represent results as functions of the bitrate generated by the UDP STA
---
- Start the UDP flow at 1Mbit/s and increase it by 1Mbit/s steps until the link capacity is clearly exceeded (i.e., the UDP throughput at the sink node will stabilize, independently of how much more traffic is offered at the sender because it is lost).
- When analysing the results, the node with ip=10.0.0.3 is the node sending the UDP flow.

The default settings are:
> `duration=10` ($s$)
>
> `udp_data_rate=1Mbps` 

You may change the parameters directly in the code and run the scenario without any argument:

```./waf --run "scratch/third_study"```

or feed the duration of the TCP and UDP flows ($s$) and the UDP data rate via command line arguments:

```./waf --run "scratch/third_study --duration=60  --udp_data_rate=1Mbps"```


In [ ]:
import subprocess
import re
import pandas as pd
import numpy as np

# Defining variables to use for the study
default = True


print('--- Third Study ---\n')

if(default): # 
    start = 1
    stop = 15
    step = 1
    duration = 1
else:
    start = input('Limite Min?')
    stop = input('Limite Max?')
    step = input('Passo?')
    duration = input('Duracao?')
    
udpdr = start
regex = r"       Throughput: (\d+\.\d+) Kbps"
rx = re.compile(regex)
#df = pd.DataFrame(columns=['Flow1','Flow2','Distance'],dtype= np.float64)
for udpr in range(start,stop+step,step):
    i = int (distance- start)/step 
    print(f' # {i} -_Duration is {duration}m, UDP Data Rates {udpdr}Mbps, running command:\n    ./waf --run "scratch/third_study --duration={duration} --udp_data_rate={udpdr}Mbps"')
    process = subprocess.run([f'./waf --run "scratch/third_study --duration={duration} --udp_data_rate={udpdr}Mbps"'], shell=True, check=True, stdout=subprocess.PIPE, universal_newlines=True)
    output = rx.findall(process.stdout)
    print(process.stdout)
    print(output)
    #df.loc[i] = output+[distance]
    #print(f'Throughput Results:\n    Flow 1 -> {output[0]} Kbps\n    Flow 2 -> {output[1]} Kbps\n\n')


## __Fourth Study – Throughput vs distance with relay__

---
### 1 AP, 1 RELAY, 1 STA, positioned in a line
---
- AP and the STA have full TCP/IP stacks
- STA generates one TCP flow towards the AP
    - The flow should be sent via the RELAY node
- Carry out simulations for different distances d, considering
    - `DistanceAP-STA= d |    DistanceAP-RELAY = DistanceRELAY-STA = d/2`
- For each distance measure the average throughput obtained by the flow
    - Throughput (in bit/s) measured at the AP
- Represent results as a function of distance
- Compare the results with the results obtained in the First Study
- In which situations should a relay node be used? Why?
---
- You may start at 100m. Lower distances result in the same effective TCP throughput.
- The maximum `distance` between the sender and the sink is expected to be around 2900m. At 3000m there is no connectivity.
- You may collect results with steps of 100m between 100m and 3200m. Decrease the step size if more resolution is needed for the resulting plot($s$).
- Compare the results with the ones obtained in the first study. Place them side by side in the same plot and elaborate some conclusions.

The default settings are:
> `duration=10` ($s$)
>
> `distance=100` ($m$)

You may change the parameters directly in the code and run the scenario without any argument:

`./waf --run "scratch/fourth_study`

or feed the `distance` ($m$) between sender and sink nodes (relay is placed at `distance/2`) and the duraton of the flow ($s$) you want to simulate via command line arguments:

`./waf --run "scratch/fourth_study --distance=100 --duration=60"`

In [3]:
import subprocess
import re
import pandas as pd
import numpy as np
import plotly.express as px
# Defining variables to use for the study
default = True


print('--- Fourth Study ---\n')

if(default): # 
    start = 100
    stop = 3000
    step = 100
    duration = 1
else:
    start = input('Limite Min?')
    stop = input('Limite Max?')
    step = input('Passo?')
    duration = input('Duracao?')

distance = start 

cmd = f'./waf --run "scratch/fourth_study --distance={distance} --duration={duration}"'
regex = r"       Throughput: (\d+\.\d+) Kbps"
rx = re.compile(regex)
df = pd.DataFrame(columns=['Flow1','Flow2','Distance'],dtype= np.float64)
for distance in range(start,stop,step):
    i = int (distance - start)/step 
    print(f'#{i} Distance is {distance}m, duration {duration}, running command:\n    ./waf --run "scratch/fourth_study --distance={distance} --duration={duration}"' )
    process = subprocess.run([f'./waf --run "scratch/fourth_study --distance={distance} --duration={duration}"'], shell=True, check=True, stdout=subprocess.PIPE, universal_newlines=True)
    output = rx.findall(process.stdout)
    df.loc[i] = output+[distance]
    print(f'Throughput Results:\n    Flow 1 -> {output[0]} Kbps\n    Flow 2 -> {output[1]} Kbps\n\n')

fig = px.line(df, x="Distance", y=["Flow1","Flow2"], title='Throughput vs Distance')
fig.show()

--- Fourth Study ---

#0.0 Distance is 100m, duration 1, running command:
    ./waf --run "scratch/fourth_study --distance=100 --duration=1"
Throughput Results:
    Flow 1 -> 11228.8 Kbps
    Flow 2 -> 197.522 Kbps


#1.0 Distance is 200m, duration 1, running command:
    ./waf --run "scratch/fourth_study --distance=200 --duration=1"
Throughput Results:
    Flow 1 -> 10972.4 Kbps
    Flow 2 -> 193.063 Kbps


#2.0 Distance is 300m, duration 1, running command:
    ./waf --run "scratch/fourth_study --distance=300 --duration=1"
Throughput Results:
    Flow 1 -> 11231.2 Kbps
    Flow 2 -> 197.585 Kbps


#3.0 Distance is 400m, duration 1, running command:
    ./waf --run "scratch/fourth_study --distance=400 --duration=1"
Throughput Results:
    Flow 1 -> 7581.37 Kbps
    Flow 2 -> 133.725 Kbps


#4.0 Distance is 500m, duration 1, running command:
    ./waf --run "scratch/fourth_study --distance=500 --duration=1"
Throughput Results:
    Flow 1 -> 5583.67 Kbps
    Flow 2 -> 98.8276 Kbps


#5.

## Results to deliver
- Report (maximum 4 pages – 1 page per study, 11pt) describing the experiments and the results
### Confidence intervals
- Simulations studies should always present confidence intervals.  The confidence intervals are obtained by running the same simulations multiple times using different runs or seeds
- But, in this project, we will accept the results obtained from a single run for each experiment, provided that you generate a large number of packets. But please note that when we use queues, packet delays and packet losses are not independent from each other, so the obtained random sequences will not be IID, and we will be unable to characterize the confidence intervals for the estimated means and estimated probabilities